In [ ]:
%pip install peft

In [ ]:
%pip install accelerate

In [ ]:
%pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
%pip install datasets

In [ ]:
%pip install "fsspec>=2021.10"

In [ ]:
%pip install "s3fs>=2021.10"

In [ ]:
%pip install transformers==4.31.0

In [ ]:
%pip install transformers[sentencepiece]
%pip install sentencepiece

In [ ]:
#%pip install transformers==4.36.0

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp.git
!git clone https://github.com/IlyaGusev/rulm.git

In [ ]:
%pip uninstall xxhash 

In [1]:
print('Hello my friend')

Hello my friend


In [1]:
print('Ayoooo')

Ayoooo


In [1]:
from transformers import Trainer

2024-05-21 22:08:38.882228: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import torch
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

In [3]:
import torch.nn.functional as F
from datasets import load_dataset
import time
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path

In [4]:
print('Ayoooo')

Ayoooo


In [5]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [6]:
print('Ayoooo')

Ayoooo


In [ ]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

In [ ]:
print('helo')

In [ ]:
#print(torch.__version__)

In [ ]:
#%pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
#!python3 -m pip install --upgrade pip

In [ ]:
#%pip install --upgrade pip

In [ ]:
%%time
inp = 'Почему трава зеленая?'
conversation = Conversation(system_prompt='Отвечай')
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print(inp)
print(output)

-------------------------------------------------
-------------------------------------------------

In [7]:
import torch
import pandas as pd


In [8]:
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig


In [10]:
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training


In [11]:
import torch.nn.functional as F
from datasets import load_dataset
import time


In [12]:
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path

In [59]:
MODEL_NAME = "IlyaGusev/saiga2_7b_lora"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

st_time = time.time()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant=False,
)


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit = True,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable = True
).to(device)

if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True
    print('Count: ', torch.cuda.device_count())

model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
print(f'Прошло времени {time.time() - st_time}')

/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.16s/it]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

Прошло времени 8.824531316757202


In [14]:
device

device(type='cuda')

In [15]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [16]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='TheBloke/Llama-2-7B-fp16', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules={'o_proj', 'k_proj', 'v_proj', 'q_proj'}, lora_alpha=16, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [17]:
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [18]:
%%time
inp = 'Почему трава зеленая?'
conversation = Conversation(system_prompt='Отвечай')
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print(inp)
print(output)

Почему трава зеленая?
Зеленый цвет травы связан с наличием в ней хлорофиллов. Хлорофилл - это пигмент, который обеспечивает защиту растения от ультрафиолетового излучения солнца и способствует фотосинтезу. Зельное окрашивание травы происходит благодаря тому, что хлорофиллам удается сохранять свои свойства при дневных температурах воздуха выше 20-30 градусов Цельсия (в зависимости от видов), тогда как другие пигменты быстро разрушаются под действием тепла. Это позволяет травянистым растениям выживать в условиях жаркого климата и получать необходимость энергии для роста и развития.
CPU times: user 15.2 s, sys: 112 ms, total: 15.3 s
Wall time: 15.3 s


--------------------

In [19]:
from datasets import load_dataset

In [20]:
data = load_dataset(
    "json", 
    data_files={'train' : '/home/jupyter/datasphere/project/val_real.json' , 'validation' : '/home/jupyter/datasphere/project/val_real.json', 'test' : '/home/jupyter/datasphere/project/test.json'}
)

data

Generating train split: 100 examples [00:00, 6582.29 examples/s]
Generating validation split: 100 examples [00:00, 19007.13 examples/s]
Generating test split: 1045 examples [00:00, 28222.37 examples/s]


DatasetDict({
    train: Dataset({
        features: ['bot', 'system', 'user'],
        num_rows: 100
    })
    validation: Dataset({
        features: ['bot', 'system', 'user'],
        num_rows: 100
    })
    test: Dataset({
        features: ['bot', 'system', 'user'],
        num_rows: 1045
    })
})

In [21]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.decode([1]))
print(tokenizer.decode([2]))

1
2
<s>
</s>


In [22]:
print(tokenizer('system', add_special_tokens=False))
print(tokenizer('user', add_special_tokens=False))
print(tokenizer('bot', add_special_tokens=False))

{'input_ids': [1788], 'attention_mask': [1]}
{'input_ids': [1404], 'attention_mask': [1]}
{'input_ids': [9225], 'attention_mask': [1]}


In [23]:
tokenizer('<s>system привет как дела</s><s>',truncation=True,
        max_length=3584,
        padding=False,
        return_tensors=None)

{'input_ids': [1, 1, 1788, 1695, 7616, 5413, 24280, 2, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [24]:
CUTOFF_LEN = 3584

SYSTEM = tokenizer('system', add_special_tokens=False)['input_ids'][0]
USER = tokenizer('user', add_special_tokens=False)['input_ids'][0]
BOT = tokenizer('bot', add_special_tokens=False)['input_ids'][0]
print(f'system: {SYSTEM}')
print(f'user: {USER}')
print(f'bot: {BOT}')

system: 1788
user: 1404
bot: 9225


In [25]:
CUTOFF_LEN = 3584

def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}</s>"""
    #     print(promt)
    return promt
 
    
def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
        
        
    
    result["labels"] = result["input_ids"].copy()

    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
#     print(tokenized_full_prompt)
    return tokenized_full_prompt

In [26]:
print(generate_and_tokenize_prompt(data["train"][0]))

{'input_ids': [1, 1, 1788, 13, 30054, 29972, 29942, 753, 717, 1097, 7678, 3378, 3327, 18077, 494, 490, 9116, 1155, 730, 4390, 1866, 614, 7832, 1587, 606, 614, 29969, 5206, 6052, 23092, 24871, 665, 8487, 27268, 29964, 6188, 1502, 29889, 939, 1685, 7616, 2771, 1521, 956, 1097, 7678, 3378, 3327, 18077, 494, 606, 11244, 6253, 12395, 29889, 2, 1, 1404, 13, 30038, 29969, 5206, 6195, 29901, 6083, 29960, 5460, 379, 3357, 26599, 350, 29941, 29896, 29896, 29899, 29906, 29906, 29896, 531, 1447, 9518, 25163, 2082, 21944, 30002, 8150, 14519, 12087, 2082, 718, 1325, 29959, 10766, 676, 13, 30038, 7832, 1755, 29901, 29871, 2014, 4222, 24178, 7372, 2370, 16042, 11744, 29901, 14570, 29899, 18800, 29899, 576, 29960, 5460, 718, 1447, 9518, 25163, 3162, 14519, 12087, 477, 29871, 29945, 757, 718, 20468, 29899, 15266, 676, 29889, 6083, 29960, 5460, 26780, 19945, 29932, 3807, 21656, 717, 317, 7833, 29899, 15266, 3327, 606, 20302, 1604, 4449, 2100, 1499, 1413, 25827, 24050, 29899, 1268, 29957, 8776, 12854, 144

In [27]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

test_data = (
    data["test"].map(generate_and_tokenize_prompt)
)

Map: 100%|██████████| 1045/1045 [00:02<00:00, 432.64 examples/s]


-----------------------

In [28]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [29]:
BATCH_SIZE = 4
MICRO_BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 200
OUTPUT_DIR = "/home/jupyter/datasphere/project/tmp/model"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            logging_strategy = "steps",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=20,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [30]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [51]:

checkpoint_file = '/home/jupyter/datasphere/project/tmp/checkpoint-100'

In [62]:
from accelerate import load_checkpoint_and_dispatch

checkpoint_file = '/home/jupyter/datasphere/project/tmp'

model = load_checkpoint_and_dispatch(
    model, checkpoint=checkpoint_file, device_map="auto"
)

ValueError: /home/jupyter/datasphere/project/tmp is not a folder containing a `.index.json` file or a pytorch_model.bin or a model.safetensors file

In [46]:
val_data[0]['system']

'Извлеки атрибуты товара в формате json из описания и оглавления продукта на маркетплейсе. В ответ выведи атрибуты товара и их значения.'

In [60]:
%%time
inp = val_data[3]['user']
conversation = Conversation(system_prompt=val_data[3]['system'])
conversation.add_user_message(inp)
prompt = conversation.get_prompt(tokenizer)

output = generate(model, tokenizer, prompt, generation_config)
print(inp)
print(output)

Оглавление: Смартфон OPPO F5 CPH1723 6/128 GB (золотистый)
Описание: Смартфон OPPO F5 CPH1723 получил 6-дюймовый дисплей, который позволяет просматривать фильмы, читать книги и находить нужную информацию в интернете, не напрягая глаз. Несмотря на это, он остался легким, компактным и очень удобным в использовании. Причиной тому стали ультратонкие рамки, которые остаются практически незаметными. К числу прогрессивных решений стоит отнести и соотношение сторон 2:1. Оно позволяет запускать два приложения одновременно, не отрываясь от просмотра фильма при отправке сообщения или загрузке файлов.2.5D стекло добавляет дополнительную эстетику и ощущение удобства при использовании Производительность OPPO F5 CPH1723 обеспечивается восьмиядерным процессором MediaTek MT6763T с тактовой частотой 2.5 ГГц. С 6 ГБ оперативной памяти и встроенной памятью объемом 128 ГБ, у вас есть достаточно ресурсов для плавной работы приложений и хранения данных.Основная камера с разрешением 16 Мпикс и встроенной вспы

In [56]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)

AttributeError: 'function' object has no attribute 'to'

In [54]:
trainer.train('/home/jupyter/datasphere/project/tmp/checkpoint-170')

ValueError: Can't find a checkpoint index (pytorch_model.bin.index.json or model.safetensors.index.json) in /home/jupyter/datasphere/project/tmp/checkpoint-170.

In [55]:
model.save_pretrained('/home/jupyter/datasphere/project/tmp/model')

AttributeError: 'function' object has no attribute 'save_pretrained'

In [ ]:
#from accelerate import load_checkpoint_and_dispatch

#model = load_checkpoint_and_dispatch(
#    model, checkpoint=checkpoint_file, device_map="auto"
#)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model = torch.compile(model)
trainer.train()
model.save_pretrained(OUTPUT_DIR)

In [ ]:
from huggingface_hub import notebook_login
 
notebook_login()
model.push_to_hub("BaronFonMonc/Saiga_2_7b_fine_tune_wb", use_auth_token=True)

---------------------------------

In [ ]:
self_instruct_dir = '/home/jupyter/datasphere/project/tmp/rulm/self_instruct'

checkpoint = "/home/jupyter/datasphere/project/tmp/checkpoint-60"

merged_model_name = '/home/jupyter/datasphere/project/tmp/merged_test_model.pt'

self_instruct_dir

In [ ]:
%cd {self_instruct_dir}

In [ ]:
%ls

In [ ]:
%pip install fire

In [ ]:
%%python 
python -m src.tools.convert_to_native {checkpoint} {merged_model_name} --device=cuda --enable_offloading

In [ ]:
merged_model_name.endswith('.pt')

In [ ]:
import os
os._exit(00) # Перезапустить ядро

In [ ]:
self_instruct_dir = '/home/jupyter/datasphere/project/tmp/rulm/self_instruct'

checkpoint = "/home/jupyter/datasphere/project/tmp/checkpoint-60"

merged_model_name = '/home/jupyter/datasphere/project/tmp/merged_test_model.pt'


In [ ]:
%cd {self_instruct_dir}

In [ ]:
!python3 -m src.tools.convert_to_native '/home/jupyter/datasphere/project/tmp/checkpoint-60' '/home/jupyter/datasphere/project/tmp/merged_test_model.pt' --device=cuda --enable_offloading

In [ ]:
print('hello')

In [ ]:
!python -m src.tools.convert_to_native {checkpoint} {merged_model_name} --device=cuda --enable_offloading
# assert (output_dir / merged_model_name).exists()

In [ ]:
%cd /home/jupyter/datasphere/project/tmp
%ls

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
tokenizer.save_pretrained('/home/jupyter/datasphere/project/tmp/checkpoint-60')

In [ ]:
%cd /home/jupyter/datasphere/project/tmp/llama.cpp

In [ ]:
#%%capture
!python3 -m pip install -r requirements.txt

In [ ]:
%pwd

In [ ]:
self_instruct_dir = '/home/jupyter/datasphere/project/tmp/rulm/self_instruct'

checkpoint = "/home/jupyter/datasphere/project/tmp/checkpoint-60"

merged_model_name = '/home/jupyter/datasphere/project/tmp/merged_test_model.pt'


In [ ]:
model_dir = '/home/jupyter/datasphere/project/tmp/merged_test_model.pt'
checkpoint = "/home/jupyter/datasphere/project/tmp/checkpoint-60"
output_model = "/home/jupyter/datasphere/project/tmp/model-f16.gguf"

In [ ]:
!python3 convert.py '/home/jupyter/datasphere/project/tmp/merged_test_model.pt' --vocab-dir  '/home/jupyter/datasphere/project/tmp/checkpoint-60' --outfile '/home/jupyter/datasphere/project/tmp/model-f16.gguf' --outtype f16 --ctx 4096

In [ ]:
!python convert.py {model_dir} --vocab-dir {checkpoint} --outfile {output_model} --outtype f16 --ctx 4096

In [ ]:
print(len(tokenizer.get_vocab()))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("IlyaGusev/saiga2_7b_lora", use_fast=False)
print(len(tokenizer.get_vocab()))

In [ ]:
%cd /kaggle/tmp/
%ls

In [ ]:
self_instruct_dir = '/home/jupyter/datasphere/project/tmp/rulm/self_instruct'

checkpoint = "/home/jupyter/datasphere/project/tmp/checkpoint-60"

merged_model_name = '/home/jupyter/datasphere/project/tmp/merged_test_model.pt'

-----------------------

In [ ]:
%cd /home/jupyter/datasphere/project/tmp/llama.cpp

In [ ]:
!make quantize

In [ ]:
model_gguf = "/home/jupyter/datasphere/project/tmp/model-f16.gguf"
quant_model = "/home/jupyter/datasphere/project/tmp/model-q4_0.gguf"
quantization_type = "q4_0" #@param ["q4_0", "q4_1"] {allow-input: true}

In [ ]:
! ./quantize '/home/jupyter/datasphere/project/tmp/model-f16.gguf' '/home/jupyter/datasphere/project/tmp/model-q4_0.gguf' 'q4_0'

In [ ]:
%cd ~
%cd /home/jupyter/datasphere/project/tmp/llama.cpp
%pwd

----------------------------

In [ ]:
%cd /home/jupyter/datasphere/project/tmp/
!git clone --recursive https://github.com/ggerganov/llama.cpp.git


In [ ]:
%cd /home/jupyter/datasphere/project/tmp/llama.cpp
!make LLAMA_CUBLAS=1 -j libllama.so

In [ ]:
# HACK: Use custom compiled libllama.so
%cp ~/llama.cpp/libllama.so /home/jupyter/datasphere/project/tmp/libllama.so

In [ ]:
# import fire
from llama_cpp import Llama
from tqdm import tqdm

In [ ]:
SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

top_k=40
top_p=0.5
temperature=0.01
repeat_penalty=1.1


ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def chat_saiga(message, model):
    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    # model.eval(tokens)
    
    message_tokens = get_message_tokens(model=model, role="user", content=message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    # print(tokens)
    # detokenize = model.detokenize(tokens)
    # print(model.tokenize(full_prompt))
    generator = model.generate(
        tokens,
        top_k = top_k,
        top_p = top_p,
        temp = temperature,
        repeat_penalty = repeat_penalty,
        reset = True
    )
    # print(len([token for token in generator]))
    
    result_list = []
    
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        print(token_str, end="", flush=True)
        result_list.append(token_str)
    return ''.join(result_list)

In [ ]:
%ls /home/jupyter/datasphere/project/tmp/llama.cpp/

In [ ]:
%cd /home/jupyter/datasphere/project/tmp
%ls

In [ ]:
%cd /home/jupyter/datasphere/project/tmp
%ls

In [ ]:
%cp ~/llama.cpp/libllama.so /home/jupyter/datasphere/project/tmp/libllama.so123

In [ ]:
%%capture
try:
    del model
except:
    pass

# model_path = '/kaggle/working/model-q4_0.gguf'
model_path = '/kaggle/working/model-q4_0.gguf'
n_ctx = 3096 #

model = Llama(
        model_path = model_path,
        n_ctx = n_ctx,
        n_gpu_layers=-1
) # n_gpu_layers=-1  параметр для переноса модели на GPU

In [ ]:
%%time
promt = """
Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Машинное обучение (ML) — это использование математических моделей данных, которые помогают компьютеру обучаться без непосредственных инструкций.
Оно считается одной из форм искусственного интеллекта (ИИ).
"""
chat_saiga(promt, model)

-------------------------------------

In [ ]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Conversation:
    def __init__(
        self,
        message_template = DEFAULT_MESSAGE_TEMPLATE,
        system_prompt = DEFAULT_SYSTEM_PROMPT,
        response_template = DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()



def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt,
                     return_tensors="pt",
                     add_special_tokens=False,
#                      padding=True,
#                     truncation=True
                    )
    data = {k: v.to(device) for k, v in data.items()}
    
    output_ids = model.generate(
        **data,
        generation_config = generation_config,
#         remove_invalid_values = True
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [ ]:
%%time
def get_message(inputs):
    result = []
    for inp in inputs:
        conversation = Conversation()
        conversation.add_user_message(inp)
        prompt = conversation.get_prompt(tokenizer)
        print(prompt)
        output = generate(model, tokenizer, prompt, generation_config)
        result.append(output)
    return result

In [ ]:
p = """
Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Линейная модель — модель, отображающая состояние или функционирование системы таким образом, что все взаимозависимости в ней принимаются линейными (см. Линейная зависимость, Линейность в экономике). Соответственно, она может формулироваться в виде одного линейного уравнения или системы линейных уравнений.
"""
get_message([p])

In [ ]:
from huggingface_hub import notebook_login
 
notebook_login()

In [ ]:
from huggingface_hub import notebook_login
 
notebook_login()
model.push_to_hub("BaronFonMonc/Saiga_2_7b_fine_tune_wb", use_auth_token=True)